# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [42]:
# Load the libraries as required.
import pandas as pd
fires_dt = pd.read_csv('../../05_src/data/fires/forestfires.csv')




In [43]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [44]:
X = fires_dt.drop('area', axis=1)


In [45]:
y = fires_dt['area']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer

# Column lists
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Preproc1: Scale + OneHot
preproc1 = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [47]:
# Preproc2: Transform (e.g., PowerTransformer) + Scale + OneHot
preproc2 = ColumnTransformer([
    ('num', Pipeline([
        ('transform', PowerTransformer()),
        ('scale', StandardScaler())
    ]), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [48]:
# Pipeline A = preproc1 + baseline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

pipe_a = Pipeline([('preprocessing', preproc1), ('regressor', Ridge())])

In [49]:
# Pipeline B = preproc2 + baseline
pipe_b = Pipeline([('preprocessing', preproc2), ('regressor', Ridge())])

In [50]:
# Pipeline C = preproc1 + advanced model
pipe_c = Pipeline([('preprocessing', preproc1), ('regressor', RandomForestRegressor(random_state=42))])

In [51]:
# Pipeline D = preproc2 + advanced model
pipe_d = Pipeline([('preprocessing', preproc2), ('regressor', RandomForestRegressor(random_state=42))]) 

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [52]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

pipe_a = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', Ridge())
])

param_grid_a = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_search_a = GridSearchCV(pipe_a, param_grid_a, cv=5, scoring='neg_root_mean_squared_error')
grid_search_a.fit(X, y)

print("Pipeline A best RMSE:", -grid_search_a.best_score_)
print("Pipeline A best params:", grid_search_a.best_params_)


Pipeline A best RMSE: 52.25257430086576
Pipeline A best params: {'regressor__alpha': 10}


In [53]:
pipe_b = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge())
])

param_grid_b = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

grid_search_b = GridSearchCV(pipe_b, param_grid_b, cv=5, scoring='neg_root_mean_squared_error')
grid_search_b.fit(X, y)

print("Pipeline B best RMSE:", -grid_search_b.best_score_)
print("Pipeline B best params:", grid_search_b.best_params_)


Pipeline B best RMSE: 52.38806278331724
Pipeline B best params: {'regressor__alpha': 10}


In [54]:
from sklearn.ensemble import RandomForestRegressor

pipe_c = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_grid_c = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [3, 5]
}

grid_search_c = GridSearchCV(pipe_c, param_grid_c, cv=5, scoring='neg_root_mean_squared_error')
grid_search_c.fit(X, y)

print("Pipeline C best RMSE:", -grid_search_c.best_score_)
print("Pipeline C best params:", grid_search_c.best_params_)


Pipeline C best RMSE: 55.52245621979063
Pipeline C best params: {'regressor__max_depth': 3, 'regressor__n_estimators': 50}


In [55]:
pipe_d = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_grid_d = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [3, 5]
}

grid_search_d = GridSearchCV(pipe_d, param_grid_d, cv=5, scoring='neg_root_mean_squared_error')
grid_search_d.fit(X, y)

print("Pipeline D best RMSE:", -grid_search_d.best_score_)
print("Pipeline D best params:", grid_search_d.best_params_)


Pipeline D best RMSE: 55.60200791302809
Pipeline D best params: {'regressor__max_depth': 3, 'regressor__n_estimators': 50}


In [56]:
results = {
    "A": -grid_search_a.best_score_,
    "B": -grid_search_b.best_score_,
    "C": -grid_search_c.best_score_,
    "D": -grid_search_d.best_score_,
}

best_model_id = min(results, key=results.get)
print(f"Best model is Pipeline {best_model_id} with RMSE: {results[best_model_id]}")


Best model is Pipeline A with RMSE: 52.25257430086576


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [57]:
results = {
    "Pipeline A (Preproc1 + Ridge)": -grid_search_a.best_score_,
    "Pipeline B (Preproc2 + Ridge)": -grid_search_b.best_score_,
    "Pipeline C (Preproc1 + RF)": -grid_search_c.best_score_,
    "Pipeline D (Preproc2 + RF)": -grid_search_d.best_score_,
}

best_model = min(results, key=results.get)
print("Best Model:", best_model)
print("Best RMSE:", results[best_model])



Best Model: Pipeline A (Preproc1 + Ridge)
Best RMSE: 52.25257430086576


In [58]:
import joblib
joblib.dump(grid_search_a.best_estimator_, '../../05_src/data/best_forest_model.pkl')


['../../05_src/data/best_forest_model.pkl']

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [59]:
import pandas as pd
import numpy as np
import shap


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
X = pd.DataFrame(X, columns=feature_names)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_model = RandomForestClassifier(random_state=42)
best_model.fit(X_train, y_train)




RandomForestClassifier(random_state=42)

In [60]:
import numpy as np
import pandas as pd
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

if not hasattr(np, 'bool'):
    np.bool = bool

X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
X = pd.DataFrame(X, columns=feature_names)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

explainer = shap.TreeExplainer(model)

shap_values_train = explainer.shap_values(X_train)

shap.summary_plot(shap_values_train[1], X_train, plot_type="bar")

feature_importance = np.abs(shap_values_train[1]).mean(axis=0)
importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
importance_df = importance_df.sort_values(by='importance', ascending=False)
print("\nFeature importance ranking:")
print(importance_df)

shap.initjs()

force_plot = shap.force_plot(explainer.expected_value[1], shap_values_train[1][0], X_train.iloc[0])
display(force_plot)

ImportError: matplotlib is not installed so plotting is not available! Run `pip install matplotlib` to fix this.


I think we could remove 'coord_x" and "coord_y' because they have very small impact and do not have meaningful result to the prediction. I can use RMSE to compare the performance and removing these can simplify the model.


## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.